In [1]:
# Reading the File in
file = open('foods.txt', 'r', encoding="ISO-8859-1")
foodfile = file.read()
file.close()

In [2]:
#Extract reviews
#Coverting all words to lower case
import re
reviews = re.findall("/text: (.*)\n", foodfile)
temp = [ ]
for i in reviews:
    #i = re.sub('<[^<]+-?>','', i)
    temp.append(i.lower()) 
reviews = temp
reviews_text = " ".join(reviews).split()

In [3]:
#unique words
L = list(set(reviews_text))
len(L)

506062

In [4]:
#reading a csv file having list of stopwords
import pandas as pd
df = pd.read_csv (r'Stopwords.csv')
#print (df)
stopwords = df['Stopwords'].tolist()

In [5]:
#Removing Stopwords and words with punctuations
W = []
for i in L:
    if i not in stopwords and i.isalpha():
        W.append(i)

In [6]:
#acceptable words
len(W)

86742

In [7]:
# calculating top 500 words
from collections import Counter
full = dict(Counter(reviews_text))
W_count = dict((k, full[k]) for k in W)
top500 = Counter(W_count).most_common()[:500]
top500words = list(map(lambda x: x[0],top500))
top500

[('good', 147853),
 ('great', 130957),
 ('taste', 127457),
 ('will', 125047),
 ('love', 121572),
 ('coffee', 115167),
 ('product', 101020),
 ('tea', 95106),
 ('flavor', 93135),
 ('food', 86883),
 ('buy', 71498),
 ('find', 69122),
 ('best', 63498),
 ('eat', 59276),
 ('dog', 58456),
 ('time', 54812),
 ('better', 54727),
 ('amazon', 52356),
 ('price', 50586),
 ('bought', 48573),
 ('chocolate', 47238),
 ('tastes', 42856),
 ('cup', 41910),
 ('drink', 41459),
 ('bag', 40613),
 ('well', 40252),
 ('bit', 40040),
 ('water', 39156),
 ('recommend', 38588),
 ('order', 38501),
 ('nice', 36066),
 ('loves', 35679),
 ('box', 35450),
 ('sweet', 35136),
 ('sugar', 34633),
 ('add', 34005),
 ('hot', 32434),
 ('cat', 32001),
 ('lot', 31898),
 ('keep', 31698),
 ('ordered', 30685),
 ('favorite', 30010),
 ('small', 29825),
 ('mix', 29435),
 ('eating', 29250),
 ('dogs', 29091),
 ('free', 28627),
 ('definitely', 28593),
 ('store', 28283),
 ('sure', 28187),
 ('buying', 28160),
 ('brand', 27534),
 ('easy', 27136)

In [8]:
reviews[0]

'i have bought several of the vitality canned dog food products and have found them all to be of good quality. the product looks more like a stew than a processed meat and it smells better. my labrador is finicky and she appreciates this product better than  most.'

In [9]:
with open('Top500Words.txt','w') as f:
      f.writelines("{}\n" .format(j) for j in top500)

In [10]:
# Passing vocabulary of top 500 words so that it can fit only among this 
from sklearn.feature_extraction.text import TfidfVectorizer
#Vectorize reviews
vectorizer = TfidfVectorizer(vocabulary = top500words, min_df=0)
vector_reviews = vectorizer.fit_transform(reviews).toarray()

In [11]:
import numpy as np

In [12]:
#Clustering the Reviews
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=26)
kmeans.fit(np.array(vector_reviews))


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=26, tol=0.0001, verbose=0)

In [13]:
#Top 5 words around each center
centers = list(kmeans.cluster_centers_)

top_words = []
values = []
for c in centers:
    ind = [list(c).index(sorted(c, reverse = True)[i]) for i in range(5)]
    words = list(vectorizer.inverse_transform(c)[0][ind])
    top_words.append(words)
    values.append(c[ind])

In [14]:
cluster = ['Cluster = '+ str(i+1) for i in range(10)]
top = pd.DataFrame({'Cluster':cluster,'Top 5 features':top_words,'Feature Values':values})
top.to_csv('top5feature.csv')
top

,Cluster,Top 5 features,Feature Values
0,Cluster = 1,"[tea, green, teas, flavor, drink]","[0.4595449963404955, 0.06225923252819412, 0.05..."
1,Cluster = 2,"[good, taste, great, flavor, love]","[0.04555091610788337, 0.045084597074933355, 0...."
2,Cluster = 3,"[coffee, cup, flavor, strong, cups]","[0.38465046833488586, 0.09591707873960156, 0.0..."
3,Cluster = 4,"[chocolate, dark, bar, taste, hot]","[0.3647019232655676, 0.06132665656398084, 0.04..."
4,Cluster = 5,"[chips, potato, bag, salt, chip]","[0.4285501675659316, 0.08482526776900509, 0.06..."
5,Cluster = 6,"[food, cat, cats, eat, dog]","[0.30508846750297103, 0.1827265686384478, 0.12..."
6,Cluster = 7,"[product, great, good, will, taste]","[0.2505921337292479, 0.06017683128238313, 0.04..."
7,Cluster = 8,"[dog, treats, dogs, treat, loves]","[0.2782460745732889, 0.16431218106478393, 0.14..."
8,Cluster = 9,"[cookies, cookie, chocolate, gluten, free]","[0.42348104251571916, 0.10352391101610292, 0.0..."
9,Cluster = 10,"[amazon, price, find, store, stores]","[0.11845581061168946, 0.10405672264699684, 0.0..."
